In [3]:
import os

SEASONS = list(range(2020,2025))

DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings") #standings directory
SCORES_DIR = os.path.join(DATA_DIR, "scores") #scores directory

In [4]:
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [8]:
#get link
async def get_html(url, selector, sleep=1, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [5]:
#scrape all months of a season 
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [9]:
await scrape_season(2022)

2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com


In [5]:
standings_files = os.listdir(STANDINGS_DIR)

In [6]:
#scrape games from html files in standings 
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get('href') for l in links]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in hrefs if l and "boxscore" in l and '.html' in l]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [9]:
import pandas as pd
#for season in SEASONS:
files = [s for s in standings_files if str(2022) in s]
    
for f in files:
    filepath = os.path.join(STANDINGS_DIR, f)
        
    await scrape_game(filepath)

Pacers vs Suns, January 22, 2022 | Basketball-Reference.com
Clippers vs Knicks, January 23, 2022 | Basketball-Reference.com
Celtics vs Wizards, January 23, 2022 | Basketball-Reference.com
Lakers vs Heat, January 23, 2022 | Basketball-Reference.com
Bulls vs Magic, January 23, 2022 | Basketball-Reference.com
Trail Blazers vs Raptors, January 23, 2022 | Basketball-Reference.com
Hawks vs Hornets, January 23, 2022 | Basketball-Reference.com
76ers vs Spurs, January 23, 2022 | Basketball-Reference.com
Grizzlies vs Mavericks, January 23, 2022 | Basketball-Reference.com
Pistons vs Nuggets, January 23, 2022 | Basketball-Reference.com
Nets vs Timberwolves, January 23, 2022 | Basketball-Reference.com
Jazz vs Warriors, January 23, 2022 | Basketball-Reference.com
Knicks vs Cavaliers, January 24, 2022 | Basketball-Reference.com
Pacers vs Pelicans, January 24, 2022 | Basketball-Reference.com
Bulls vs Thunder, January 24, 2022 | Basketball-Reference.com
Jazz vs Suns, January 24, 2022 | Basketball-Refer